In [ ]:
import warnings
warnings.filterwarnings('ignore')
from pyspark import SparkContext
sc =SparkContext()
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

from pyspark import SparkContext
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("LDA").getOrCreate()
df = spark.read.csv('8_finance_comments.csv', header=True)
df.show(10)
df.show(10, False)
df.count()
df.printSchema()


22/10/18 17:11:59 WARN Utils: Your hostname, m0 resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/10/18 17:11:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/18 17:12:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/18 17:12:06 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/18 17:12:06 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/10/18 17:12:06 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/10/18 17:12:06 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/10/18 17:12:06 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
22/10/18 17:12:06 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
22/10/18 17:12:12 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+--------------------+---------+
|      cleaned_tweets|sentiment|
+--------------------+---------+
|video offic mind ...|        0|
|price lumber lb f...|        0|
|say american drea...|       -1|
|barri silbert ext...|        1|
|satellit avoid at...|       -1|
|david butler favo...|        0|
|miss convo one fa...|        0|
|u intellig docume...|        0|
|senat want emerg ...|        1|
|hedg fund manag m...|        0|
+--------------------+---------+
only showing top 10 rows

+------------------------------------------------------------------------+---------+
|cleaned_tweets                                                          |sentiment|
+------------------------------------------------------------------------+---------+
|video offic mind busi david solomon tell gs intern learn wa             |0        |
|price lumber lb f sinc hit ytd high maci turnaround still happen        |0        |
|say american dream dead                                                 |-1       |
|b

root
 |-- cleaned_tweets: string (nullable = true)
 |-- sentiment: string (nullable = true)



In [ ]:
# for text processing
from nltk.corpus import stopwords
import re as re
from pyspark.ml.feature import CountVectorizer , IDF

# for building the model
from pyspark.mllib.linalg import Vector, Vectors
from pyspark.mllib.clustering import LDA, LDAModel

In [ ]:
# using nltk library for removing stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/hadoopuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# remove all the extra spaces between words, 
# split each review into list of words, change them into lowercase, 
# check if they’re alpha numeric, 
# remove any words or typos which are less than three letters, 
# remove any stopwords, and finally add an index to the elements.
reviews = df.rdd.map(lambda x : x['cleaned_tweets']).filter(lambda x: x is not None)
StopWords = stopwords.words("english")
tokens = reviews\
.map( lambda document: document.strip().lower()) \
.map( lambda document: re.split(" ", document)) \
.map( lambda word: [x for x in word if x.isalpha()]) \
.map( lambda word: [x for x in word if len(x) > 3] ) \
.map( lambda word: [x for x in word if x not in StopWords]).zipWithIndex()

In [ ]:
# Build a TF-IDF matrix before I run the LDA. 
# CountVectorizer takes the data and returns a sparse matrix of term frequencies 
# Same thing goes for the IDF.
df_txts = sqlContext.createDataFrame(tokens, ["list_of_words",'index'])

# TF
cv = CountVectorizer(inputCol="list_of_words", outputCol="raw_features", vocabSize=5000, minDF=10.0)
cvmodel = cv.fit(df_txts)
result_cv = cvmodel.transform(df_txts)

# IDF
idf = IDF(inputCol="raw_features", outputCol="features")
idfModel = idf.fit(result_cv)
result_tfidf = idfModel.transform(result_cv)

In [ ]:
result_tfidf.show(10,False)

+---------------------------------------------------------------------------+-----+----------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|list_of_words                                                              |index|raw_features                                                                |features                                                                                                                                                                                  |
+---------------------------------------------------------------------------+-----+----------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
from pyspark.ml.clustering import LDA
train_df = result_tfidf[['index','features']]
lda = LDA(k=5, maxIter=1)
lda_model = lda.fit(train_df)
topics = lda_model.describeTopics(5)
topics.show()
transformed = lda_model.transform(result_tfidf)
transformed.show(10)

+-----+--------------------+--------------------+
|topic|         termIndices|         termWeights|
+-----+--------------------+--------------------+
|    0|  [9, 39, 15, 4, 10]|[0.01297565482541...|
|    1|[0, 3, 125, 110, ...|[0.00742301627050...|
|    2|    [4, 7, 10, 5, 0]|[0.00719242188546...|
|    3| [26, 5, 23, 29, 12]|[0.00970338244794...|
|    4|[64, 2, 21, 911, 14]|[0.00774093233990...|
+-----+--------------------+--------------------+



+--------------------+-----+--------------------+--------------------+--------------------+
|       list_of_words|index|        raw_features|            features|   topicDistribution|
+--------------------+-----+--------------------+--------------------+--------------------+
|[video, offic, mi...|    0|(2386,[52,245,312...|(2386,[52,245,312...|[0.00407319382373...|
|[price, lumber, s...|    1|(2386,[3,15,210,3...|(2386,[3,15,210,3...|[0.71122582021024...|
|[american, dream,...|    2|(2386,[160,2038],...|(2386,[160,2038],...|[0.01473582789500...|
|[barri, silbert, ...|    3|(2386,[68,144,687...|(2386,[68,144,687...|[0.00838754294974...|
|[satellit, avoid,...|    4|(2386,[104,435,15...|(2386,[104,435,15...|[0.24286761447877...|
|[david, butler, f...|    5|(2386,[0,578,656,...|(2386,[0,578,656,...|[0.21862206332515...|
|[miss, convo, fav...|    6|(2386,[186,1872],...|(2386,[186,1872],...|[0.01462785288609...|
|[intellig, docume...|    7|(2386,[153,296,23...|(2386,[153,296,23...|[0.0102409